Basic pipeline

1. Design a model
2. Construct loss and optimizer
3. Train model
    1. forward pass: compute predictions
    2. backward pass: gradients
    3. update weights

In [28]:
import torch
import torch.nn as nn
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import datasets
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

sns.set()

In [29]:
bc = datasets.load_breast_cancer()
X, y = bc.data, bc.target

n_samples, n_features = X.shape

X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.8, random_state=1234)

sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.fit_transform(X_test)

X_train = torch.from_numpy(X_train.astype(np.float32))
X_test = torch.from_numpy(X_test.astype(np.float32))
y_train = torch.from_numpy(y_train.astype(np.float32))
y_test = torch.from_numpy(y_test.astype(np.float32))

y_train = y_train.view(y_train.shape[0], 1)
y_test = y_test.view(y_test.shape[0], 1)


In [30]:
class LogReg(nn.Module):
    def __init__(self, n_input_features) -> None:
        super().__init__()
        self.n_input_features = n_input_features
        
        self.linear = nn.Linear(n_input_features, 1)
    
    def forward(self, x):
        y_pred = torch.sigmoid(self.linear(x))
        return y_pred

In [31]:
lr = 0.01
num_epochs = 200

model = LogReg(n_features)
criterion = nn.BCELoss()
optimizer = torch.optim.SGD(model.parameters(), lr=lr)

In [32]:
for epoch in range(1, num_epochs + 1):
    y_pred = model(X_train)
    
    loss = criterion(y_pred, y_train)
    
    loss.backward()
    
    optimizer.step()
    optimizer.zero_grad()
    
    if epoch % 10 == 0:
        print(f"{epoch = }, loss = {loss.item():.3f}")

epoch = 10, loss = 0.544
epoch = 20, loss = 0.451
epoch = 30, loss = 0.391
epoch = 40, loss = 0.349
epoch = 50, loss = 0.318
epoch = 60, loss = 0.294
epoch = 70, loss = 0.275
epoch = 80, loss = 0.259
epoch = 90, loss = 0.246
epoch = 100, loss = 0.235
epoch = 110, loss = 0.225
epoch = 120, loss = 0.216
epoch = 130, loss = 0.209
epoch = 140, loss = 0.202
epoch = 150, loss = 0.195
epoch = 160, loss = 0.190
epoch = 170, loss = 0.185
epoch = 180, loss = 0.180
epoch = 190, loss = 0.176
epoch = 200, loss = 0.172


In [33]:
with torch.no_grad():
    res = model(X_test)
    y_pred = res.round()
    acc = y_pred.eq(y_test).sum() / float(y_test.shape[0])
    print(f"acc = {acc.item():.3f}")

acc = 0.930
